# Model-free Pulse Gate Calibration using Reinforcement Learning

In this notebook, we adjust the definition of the gate to be a custom pulse sequence to be simulated by Qiskit-Dynamics package.

This notebook combines previously introduced modules of Qiskit and Tensorflow, combined to the usage of Qiskit-Dynamics to handle a pulse level simulation.

In [ ]:
path = "/Users/arthurstrauss/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Coding_projects/Quantum_Optimal_Control"

import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import sys

module_path = os.path.abspath(os.path.join(path))
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
from quantumenvironment import QuantumEnvironment
from qconfig import QiskitConfig
from helper_functions import (
    select_optimizer,
    generate_model,
    get_control_channel_map,
)

# Qiskit imports for building RL environment (circuit level)
from qiskit import pulse, transpile
from qiskit.quantum_info import Operator
from qiskit.providers.options import Options
from qiskit.providers import QubitProperties, BackendV1, BackendV2

from qiskit_dynamics import DynamicsBackend, Solver
from custom_jax_sim import JaxSolver
from qiskit_dynamics.array import Array
from qiskit.circuit import ParameterVector, QuantumCircuit, QuantumRegister, Gate

# Tensorflow imports for building RL agent and framework
import tensorflow as tf
from tensorflow_probability.python.distributions import MultivariateNormalDiag
from gymnasium.spaces import Box

# Additional imports
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
from typing import Optional, Union, List

# configure jax to use 64 bit mode
import jax

jax.config.update("jax_enable_x64", True)
# tell JAX we are using CPU
jax.config.update("jax_platform_name", "cpu")
# import Array and set default backend

Array.set_default_backend("jax")

# Defining QuantumEnvironment features

We provide below the details of our custom Quantum Processing Unit (QPU) we are controlling.

## Generic information characterizing the quantum system

The algorithm is built upon Qiskit modules. To specify how to address our quantum system of interest, we therefore adopt the IBM approach to define a quantum backend, on which qubits are defined and can be accessed via control actions and measurements.

The cell below specifies:
- ```qubit_tgt_register```: List of qubit indices which are specifically addressed by controls , namely the ones for which we intend to calibrate a gate upon or steer them in a specific quantum state. Note that this list could include less qubits than the total number of qubits, which can be useful when one wants to take into account crosstalk effects emerging from nearest-neigbor coupling.
- ```sampling_Paulis```: number of Pauli observables  to be sampled from the system: the algorithm relies on the ability to process measurement outcomes to estimate the expectation value of different Pauli operators. The more observables we provide for sampling, the more properties we are able to deduce with accuracy about the actual state that was created when applying our custom controls. For a single qubit, the possible Pauli operators are $\sigma_0=I$, $\sigma_x=X$, $\sigma_y=Y$, $\sigma_z=Z$. For a general multiqubit system, the Pauli observables are tensor products of those single qubit Pauli operators. The algorithm will automatically estimate which observables are the most relevant to sample based on the provided target. The probability distribution from which those observables are sampled is derived from the Direct Fidelity Estimation (equation 3, https://link.aps.org/doi/10.1103/PhysRevLett.106.230501) algorithm. 
- ```N_shots```: Indicates how many measurements shall be done for each provided circuit (that is a specific combination of an action vector and a Pauli observable to be sampled)
- The dimension of the action vector: Indicates the number of pulse/circuit parameters that characterize our parametrized quantum circuit.
- ```estimator_options```: Options of the Qiskit Estimator primitive. The Estimator is the Qiskit module enabling an easy computation of Pauli expectation values. One can set options to make this process more reliable (typically by doing some error mitigation techniques in post-processing). Works only with Runtime Backend at the moment
- ```abstraction_level``` chosen to encode our quantum circuit. One can choose here to stick to the usual circuit model of quantum computing, by using the ```QuantumCircuit``` objects from Qiskit and therefore set the ```abstraction_level``` to ```"circuit"```. However, depending on the task at hand, one can also prefer to use a pulse description of all the operations in our circuit. This is possible by using resources of another module of Qiskit called Qiskit Dynamics. In this case, one should define the ansatz circuit above in a pulse level fashion, and the simulation done at the Hamiltonian level, and not only via statevector calculations. In this notebook we set the ```abstraction_level``` to ```"pulse"```. Another notebook at the gate level is available in the repo.

In [2]:
def custom_pulse_schedule(
    backend: Union[BackendV1, BackendV2],
    qubit_tgt_register: List[int],
    params: ParameterVector,
    default_schedule: Optional[Union[pulse.ScheduleBlock, pulse.Schedule]] = None,
):
    """
    Define parametrization of the pulse schedule characterizing the target gate
        :param backend: IBM Backend on which schedule shall be added
        :param qubit_tgt_register: Qubit register on which
        :param params: Parameters of the Schedule
        :param default_schedule:  baseline from which one can customize the pulse parameters

        :return: Parametrized Schedule
    """

    if default_schedule is None:  # No baseline pulse, full waveform builder
        pass
    else:
        # Look here for the pulse features to specifically optimize upon, for the x gate here, simply retrieve relevant
        # parameters for the Drag pulse
        pulse_ref = default_schedule.instructions[0][1].pulse

        with pulse.build(
            backend=backend, name="param_schedule"
        ) as parametrized_schedule:
            pulse.play(
                pulse.Drag(
                    duration=pulse_ref.duration,
                    amp=params[0],
                    sigma=pulse_ref.sigma,
                    beta=pulse_ref.beta,
                    angle=pulse_ref.angle,
                ),
                channel=pulse.DriveChannel(qubit_tgt_register[0]),
            )

        return parametrized_schedule

In [3]:
from typing import Dict


# Pulse gate ansatz


def apply_parametrized_circuit(
    qc: QuantumCircuit,
    params: ParameterVector,
    q_reg: QuantumRegister,
    n_actions: int,
    backend: BackendV2,
    target: Dict,
):
    """
    Define ansatz circuit to be played on Quantum Computer. Should be parametrized with Qiskit ParameterVector
    This function is used to run the QuantumCircuit instance on a Runtime backend
    :param qc: Quantum Circuit instance to add the gate on
    :return:
    """
    # qc.num_qubits

    # x_pulse = backend.defaults().instruction_schedule_map.get('x', (qubit_tgt_register,)).instructions[0][1].pulse

    # original_calibration = backend.instruction_schedule_map.get(target["name"])
    gate, physical_qubits = target["gate"], target["register"]
    parametrized_gate = Gate(
        f"custom_{gate.name}", gate.num_qubits, params=params.params
    )
    if isinstance(backend, BackendV1):
        instruction_schedule_map = backend.defaults().instruction_schedule_map
    else:
        instruction_schedule_map = backend.target.instruction_schedule_map()
    default_schedule = instruction_schedule_map.get(gate.name, physical_qubits)
    parametrized_schedule = custom_pulse_schedule(
        backend=backend,
        qubit_tgt_register=physical_qubits,
        params=params,
        default_schedule=default_schedule,
    )
    qc.add_calibration(parametrized_gate, physical_qubits, parametrized_schedule)
    qc.append(parametrized_gate, q_reg)

In [4]:
qubit_tgt_register = [0]  # Choose which qubits of the QPU you want to address
sampling_Paulis = 200
N_shots = (
    100  # Number of shots for sampling the quantum computer for each action vector
)
n_actions = 1  # Choose how many control parameters in pulse/circuit parametrization
abstraction_level = (
    "pulse"  # Choose at which abstraction level the circuit ansatz is written
)
estimator_options = {"resilience_level": 0}
batchsize = 50  # Batch size (iterate over a bunch of actions per policy to estimate expected return)
action_space = Box(
    low=-0.5, high=0.5, shape=(n_actions,), dtype=np.float64
)  # Action space for the agent
obs_space = Box(
    low=-1, high=1, shape=(1,), dtype=np.float64
)  # Observation space for the agent

Choose below which IBM Backend to use. As we are dealing with pulse level implementation, we can either simulate a backend using QiskitDynamics, or use a real backend that supports OpenPulse features.

## 1. Setting up a Quantum Backend

### Real backend initialization

Uncomment the cell below to declare a Qiskit Runtime backend. You need an internet connection and an IBM Id account to access this.

In [5]:
"""
Real backend initialization:
Run this cell only if intending to use a real backend, where Qiskit Runtime is enabled
"""

from qiskit_ibm_runtime import QiskitRuntimeService

backend_name = "ibm_perth"

# service = QiskitRuntimeService(channel='ibm_quantum')
# runtime_backend = service.get_backend(backend_name)

### Simulation backend initialization
If you want to run the algorithm over a simulation, you can rely on Qiskit Dynamics for pulse level simulation of quantum circuits. Below we set the ground for declaring a ```DynamicsBackend```.

This can be done in two ways: 

1. Declare a ```DynamicsBackend``` from a ```FakeBackend``` or ```IBMBackend``` instance and use the ```from_backend()``` method to retrieve the Hamiltonian description of such backend.
2. Alternatively, you can define your own custom Hamiltonian/Linblad that should be used to simulate the multiqubit system of interest, and feed it to a ```Solver``` instance which can be used to declare the ```DynamicsBackend```.
For more information you can check Qiskit Dynamics documentation (https://qiskit.org/documentation/dynamics/apidocs/backend.html)


#### 1. Using ```FakeBackend``` as starting point

In [31]:
from qiskit_ibm_runtime.fake_provider import FakeHanoi, FakeJakarta, FakeJakartaV2

fake_backend = FakeJakarta()
fake_backend_v2 = FakeJakartaV2()
control_channel_map = get_control_channel_map(
    fake_backend, list(range(fake_backend_v2.num_qubits))
)
dt = fake_backend_v2.target.dt

In [32]:
dynamics_options = {
    "seed_simulator": None,  # "configuration": fake_backend.configuration(),
    "control_channel_map": control_channel_map,
    # Control channels to play CR tones, should match connectivity of device
    "solver_options": {"method": "jax_odeint", "atol": 1e-6, "rtol": 1e-8, "hmax": dt},
}
dynamics_backend = DynamicsBackend.from_backend(
    fake_backend, subsystem_list=qubit_tgt_register, **dynamics_options
)

dynamics_backend.target.qubit_properties = fake_backend_v2.qubit_properties(
    qubit_tgt_register
)

# Extract channel frequencies and Solver instance from backend to provide a pulse level simulation enabling
# fidelity benchmarking
calibration_files = None

#### 2. Using a custom Hamiltonian model

In [33]:
r = 0.1e9

# Frequency of the qubit transition in GHz.
w = 5e9
# Sample rate of the backend in ns.
dt = 2.2222222e-10

drift = 2 * np.pi * w * Operator.from_label("Z") / 2
operators = [2 * np.pi * r * Operator.from_label("X") / 2]

hamiltonian_solver = JaxSolver(
    static_hamiltonian=drift,
    hamiltonian_operators=operators,
    rotating_frame=drift,
    rwa_cutoff_freq=2 * 5e9,
    hamiltonian_channels=["d0"],
    channel_carrier_freqs={"d0": w},
    dt=dt,
)

custom_backend = DynamicsBackend(hamiltonian_solver, **dynamics_options)

### Choose backend and define Qiskit config dictionary
Below, set the Backend that you would like to run among the above defined backend.
Then define the config gathering all the components enabling the definition of the ```QuantumEnvironment```.



In [34]:
# Choose backend among the set defined above: {runtime_backend, dynamics_backend, custom_backend}
backend = dynamics_backend

In [35]:
from qiskit.circuit.library import CXGate
from helper_functions import perform_standard_calibrations

cals, results = perform_standard_calibrations(backend)

## 2. Define quantum target: State preparation or Gate calibration

The target of our optimal control task can be of two different types:
1.  An arbitrary quantum state to prepare with high accuracy
2. A Quantum Gate to be calibrated in a noise-robust manner

Both targets are dictionaries that are identified with a key stating their ```target_type```, which can be either ```"state"``` or ```"gate"```.

For a gate target $G$, one can add the target quantum gate with a ```"gate"``` argument specifying a specific instance of a Qiskit ```Gate``` object. Here, we settle for calibrating a ```XGate()```.
Moreover, a gate calibration requires a set of input states $\{|s_i\rangle \}$ to be provided, such that the agent can try to set the actions such that the fidelity between the anticipated ideal target state (calculated as  $G|s_i\rangle$) and the output state are simultaneously maximized. To ensure a correlation between the average reward computed from the measurement outcomes and the average gate fidelity, the provided set of input states must be tomographically complete. Note that providing the set of ```input_states```as depicted below is optional and should be done only if you have a specific set to implement, by default it is internally set to the Pauli basis preparation stage.

For a state target, one can provide, similarly to an input state, an ideal circuit to prepare it (```"circuit": QuantumCircuit```, or a density matrix (key ```"dm": DensityMatrix```). Below, we settle for giving a ```QuantumCircuit```for each input state, from the elementary operations present in ```qiskit.opflow```(which will be deprecated soon, and should therefore be replaced in the future by the ```QuantumCircuit```itself.

Another important key that should figure in the dictionary is the ```"register"``` indicating the qubits indices that should be addressed by this target, i.e. upon which qubits should the target be engineered. The register can be a list of indices for qubits to be addressed in the circuit, or a ```QuantumRegister```object. If register is not provided, then by default the target register will be the list of all qubits defined up to ```Gate().num_qubits```.

In [36]:
from qiskit.circuit.library import XGate

# Example of target gate
X_tgt = {"gate": XGate(), "physical_qubits": qubit_tgt_register}

In [37]:
# Choose which target to use
target = X_tgt

## 3. Declare QuantumEnvironment object
Running the box below declares the QuantumEnvironment instance.

If selected backend is a ```DynamicsBackend```, this declaration launches a series of single qubit gate calibrations (to calibrate X and SX gate). The reason for this is that the Estimator primitive, which enables the easy calculation of Pauli expectation values, needs to append gates for doing Pauli basis rotations (SX and Rz gate).

In [38]:
from qconfig import QEnvConfig, ExecutionConfig, BenchmarkConfig

# Define quantum environment
execution_config = ExecutionConfig(
    n_shots=N_shots,
    batch_size=batchsize,
    n_reps=1,
    sampling_Paulis=sampling_Paulis,
    c_factor=1.0,
)

benchmark_config = BenchmarkConfig(benchmark_cycle=5)
# Wrap all backend related info in one QiskitConfig
Qiskit_setup = QiskitConfig(
    parametrized_circuit=apply_parametrized_circuit,
    backend=backend,
    parametrized_circuit_kwargs={
        "n_actions": n_actions,
        "backend": backend,
        "target": target,
    },
    calibration_files=calibration_files,
)

q_env_config = QEnvConfig(
    target=target,
    backend_config=Qiskit_setup,
    action_space=action_space,
    execution_config=execution_config,
    benchmark_config=benchmark_config,
)

In [39]:
%%time
q_env = QuantumEnvironment(q_env_config)

In [40]:
control_channel_map

In [41]:
from qiskit import transpile

transpile(q_env.circuit_truncations[0], dynamics_backend).draw(output="mpl")

# Defining the RL agent: PPO

In [42]:
"""
-----------------------------------------------------------------------------------------------------
Hyperparameters for RL agent
-----------------------------------------------------------------------------------------------------
"""

%time
n_epochs = 200  # Number of epochs
opti = "Adam"  # Optimizer choice
eta = 0.001  # Learning rate for policy update step
eta_2 = None  # Learning rate for critic (value function) update step

use_PPO = True
epsilon = 0.1  # Parameter for clipping value (PPO)
grad_clip = 0.01
critic_loss_coeff = 0.5
optimizer = select_optimizer(
    lr=eta, optimizer=opti, grad_clip=grad_clip, concurrent_optimization=True, lr2=eta_2
)
sigma_eps = 1e-3  # for numerical stability

In [43]:
"""
-----------------------------------------------------------------------------------------------------
Policy parameters
-----------------------------------------------------------------------------------------------------
"""

N_in = obs_space.shape[
    -1
]  # One input for each measured qubit state (0 or 1 input for each neuron)
hidden_units = [82, 82]  # List containing number of units in each hidden layer

network = generate_model((N_in,), hidden_units, n_actions, actor_critic_together=True)
network.summary()
init_msmt = np.zeros(
    (1, N_in)
)  # Here no feedback involved, so measurement sequence is always the same

In [44]:
# Plotting tools
plt.rcParams["figure.dpi"] = 300
plt.rcParams["savefig.dpi"] = 300
avg_return = np.zeros(n_epochs)
fidelities = np.zeros(n_epochs)
visualization_steps = 5
%matplotlib inline
do_benchmark = False

## Run algorithm

In [46]:
"""
-----------------------------------------------------------------------------------------------------
Training loop
-----------------------------------------------------------------------------------------------------
"""

%time
q_env.clear_history()
mu_old = tf.Variable(initial_value=network(init_msmt)[0][0], trainable=False)
sigma_old = tf.Variable(initial_value=network(init_msmt)[1][0], trainable=False)

for i in tqdm(range(n_epochs)):
    Old_distrib = MultivariateNormalDiag(
        loc=mu_old, scale_diag=sigma_old, validate_args=True, allow_nan_stats=False
    )
    obs, _ = q_env.reset()
    print("\n Epoch", i)

    with tf.GradientTape(persistent=True) as tape:
        mu, sigma, b = network(init_msmt, training=True)
        mu = tf.squeeze(mu, axis=0)
        sigma = tf.squeeze(sigma, axis=0)
        b = tf.squeeze(b, axis=0)

        Policy_distrib = MultivariateNormalDiag(
            loc=mu, scale_diag=sigma, validate_args=True, allow_nan_stats=False
        )

        action_vector = tf.stop_gradient(
            tf.clip_by_value(Policy_distrib.sample(batchsize), -0.5, 0.5)
        )
        # Adjust the action vector according to params physical significance
        reward = q_env.perform_action(action_vector)
        advantage = reward - b

        if use_PPO:
            ratio = Policy_distrib.prob(action_vector) / (
                tf.stop_gradient(Old_distrib.prob(action_vector)) + 1e-6
            )
            actor_loss = -tf.reduce_mean(
                tf.minimum(
                    advantage * ratio,
                    advantage * tf.clip_by_value(ratio, 1 - epsilon, 1 + epsilon),
                )
            )
        else:  # REINFORCE algorithm
            actor_loss = -tf.reduce_mean(
                advantage * Policy_distrib.log_prob(action_vector)
            )

        critic_loss = tf.reduce_mean(advantage**2)
        combined_loss = actor_loss + critic_loss_coeff * critic_loss

    grads = tape.gradient(combined_loss, network.trainable_variables)
    print("Policy params", np.array(mu), "+-", np.array(sigma))
    # For PPO, update old parameters to have access to "old" policy
    if use_PPO:
        mu_old.assign(mu)
        sigma_old.assign(sigma)

    avg_return[i] = np.mean(q_env.reward_history, axis=1)[i]
    print("Average reward", avg_return[i])
    if do_benchmark:
        if target["target_type"] == "gate":
            fidelities[i] = q_env.avg_fidelity_history[i]
            print("Average Gate fidelity", fidelities[i])
        else:
            fidelities[i] = q_env.state_fidelity_history[i]
            print("State fidelity", fidelities[i])
    if i % visualization_steps == 0:
        clear_output(wait=True)  # for animation
        fig, ax = plt.subplots()
        ax.plot(
            np.arange(1, n_epochs, visualization_steps),
            avg_return[0:-1:visualization_steps],
            "-.",
            label="Average return",
        )
        if do_benchmark:
            ax.plot(
                np.arange(1, n_epochs, visualization_steps),
                fidelities[0:-1:visualization_steps],
                label="Average Gate Fidelity",
            )
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Reward/Gate Fidelity")
        ax.legend()
        plt.show()
        if do_benchmark:
            print(
                "Maximum fidelity reached so far:",
                np.max(fidelities),
                "at Epoch",
                np.argmax(fidelities),
            )
        print(
            "Maximum return reached so far:",
            np.max(avg_return),
            "at Epoch",
            np.argmax(avg_return),
        )
        # Apply gradients
    optimizer.apply_gradients(zip(grads, network.trainable_variables))
q_env.close()

In [52]:
plt.plot(
    np.arange(0, 200, q_env.benchmark_cycle),
    q_env.avg_fidelity_history,
    label="Average gate fidelity",
)
plt.plot(np.mean(q_env.reward_history, axis=1), label="Average reward")
plt.legend()